In [1]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
# !mkdir datasets

In [3]:
# !unzip test_dataset0202.zip

In [4]:
# !mv test_dataset/ datasets/

In [5]:
# !mv decoders.yml datasets/test_dataset

# Formal

In [6]:
import yaml
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

In [7]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # size: 512x512
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 256x256
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # size: 256x256
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 128x128
            nn.Conv2d(32, 32, kernel_size=3, padding=1),  # size: 128x128
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 64x64
            nn.Flatten(),
            nn.Linear(32 * 64 * 64, 1024),  # size: 1024
        )

    def forward(self, x):
        x = self.features(x)
        return x

In [8]:
import torch.nn as nn

class ParamAwareMultiTailDecoder(nn.Module):
    def __init__(self, input_size, classification_params=None, regression_params=None, dropout_prob=0.5):
        super(ParamAwareMultiTailDecoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(1024, 512)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=dropout_prob)
        self.fc4 = nn.Linear(512, 512)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(p=dropout_prob)
        
        self.classification_tails = nn.ModuleDict(
            {
                param_name: nn.Sequential(
                    nn.Linear(512, size),
                    nn.Softmax(dim=1),
                )
                for param_name, size in classification_params.items()
            }
        ) if classification_params else {}

        self.regression_tail = nn.ModuleDict(
            {
                param_name: nn.Sequential(
                    nn.Linear(512, 512),
                    nn.ReLU(),
                    nn.Dropout(p=dropout_prob),
                    nn.Linear(512, 512),
                    nn.ReLU(),
                    nn.Dropout(p=dropout_prob),
                    nn.Linear(512, 512),
                    nn.ReLU(),
                    nn.Dropout(p=dropout_prob),
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Dropout(p=dropout_prob),
                    nn.Linear(256, 128),
                    nn.ReLU(),
                    nn.Dropout(p=dropout_prob),
                    nn.Linear(128, size),
                )
                for param_name, size in regression_params.items()
            }
        ) if regression_params else {}

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.dropout4(x)
        classification_outputs = {
            param_name: tail(x) for param_name, tail in self.classification_tails.items()
        } if self.classification_tails else {}
        regression_output = {
            param_name: tail(x) for param_name, tail in self.regression_tail.items()
        } if self.regression_tail else {}
        return classification_outputs, regression_output


In [9]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, encoder, decoders):
        super(EncoderDecoderModel, self).__init__()
        self.encoder = encoder
        self.decoders = decoders

    def forward(self, x):
        x = self.encoder(x)
        batch_size = x.size(0)  # Get the batch size
        x = x.view(batch_size, -1)  # Flatten the feature tensor, considering the batch size
        decoder_outputs = {decoder_name: decoder(x) for decoder_name, decoder in self.decoders.items()}
        return decoder_outputs  # note that the multi-tail decoder returns a list of outputs

In [10]:
class DAGDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_name: str, datasets_folder: str="./datasets", transform=None, device=None):
        self.dataset_name = dataset_name
        self.datasets_folder = datasets_folder
        self.dataset_path = os.path.join(self.datasets_folder, self.dataset_name)
        self.images_folder = os.path.join(self.dataset_path, "images")
        self.params_folder = os.path.join(self.dataset_path, "params")
        self.ranges_file_path = os.path.join(self.dataset_path, "ranges.yml")
        self.ranges = None
        self.decoders = None
        self.transform = transforms.Compose(
            [transforms.Resize((512, 512)), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
            ) if transform is None else transform
        self.data = self.load_data()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device

    def load_data(self):
        # Load the ranges from the YAML file
        with open(self.ranges_file_path, 'r') as file:
            self.ranges = yaml.safe_load(file)
        # Load the decoders from the YAML file
        with open(os.path.join(self.dataset_path, "decoders.yml"), 'r') as file:
            self.decoders = yaml.safe_load(file)
        # read images and parameters
        data = []
        for image_name in os.listdir(self.images_folder):
            image_path = os.path.join(self.images_folder, image_name)
            param_path = os.path.join(self.params_folder, os.path.splitext(image_name)[0] + ".yml")
            with open(param_path, 'r') as file:
                param = yaml.safe_load(file)
            # normalize
            param = self.preprocess(param)
            param = self.format_target_to_decoders(param)
            data.append((image_path, param))
        return data

    def format_target_to_decoders(self, target):
        formatted_target = {}
        for decoder_name, decoder_params in self.decoders.items():
            formatted_target[decoder_name] = {
                "classification_targets": {},
                "regression_target": {}
            }
            for param_name in decoder_params:
                param_type = self.ranges[param_name]['type']
                if param_type == 'float' or param_type == 'int' or param_type == 'vector':
                    formatted_target[decoder_name]['regression_target'][param_name] = target[param_name]
                elif param_type == 'states' or param_type == 'bool':
                    formatted_target[decoder_name]['classification_targets'][param_name] = target[param_name]
        return formatted_target

    def preprocess(self, param):
        processed_param = {}
        # for float and vector: normalize with min max
        # for states, bool: convert to one hot
        # for ints: treat as float, but round back to int when saving as param
        for param_name, param_spec in self.ranges.items():
            if param_spec['type'] == 'float' or param_spec['type'] == 'int' or param_spec['type'] == 'vector':
                processed_param[param_name] = self.normalize(param[param_name], param_spec)
            elif param_spec['type'] == 'states' or param_spec['type'] == 'bool':
                processed_param[param_name] = self.one_hot(param[param_name], param_spec)
            else:
                raise ValueError(f"Unsupported parameter type: {param_spec['type']}")
        return processed_param

    def normalize(self, value, param_spec):
        if param_spec['type'] == 'float' or param_spec['type'] == 'int':
            return (value - param_spec['min']) / (param_spec['max'] - param_spec['min'])
        elif param_spec['type'] == 'vector':
            return [(value[i] - param_spec[f'{dim}min']) / (param_spec[f'{dim}max'] - param_spec[f'{dim}min']) for i, dim in enumerate(['x', 'y', 'z'])]
        else:
            raise ValueError(f"Unsupported parameter type: {param_spec['type']}")

    def one_hot(self, value, param_spec):
        if param_spec['type'] == 'states':
            index = param_spec['values'].index(value)
            return [1 if i == index else 0 for i in range(len(param_spec['values']))]
        elif param_spec['type'] == 'bool':
            # make bools onehot too to make it consistent
            return [1, 0] if value else [0, 1]
        else:
            raise ValueError(f"Unsupported parameter type: {param_spec['type']}")


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample, target = self.data[idx]
        sample = Image.open(sample).convert('L')

        if self.transform:
            sample = self.transform(sample)

        # convert target's values to tensor
        for decoder_name, decoder_outputs in target.items():
            # check if is tensor
            for classification_target in decoder_outputs['classification_targets']:
                if not torch.is_tensor(decoder_outputs['classification_targets'][classification_target]):
                    target[decoder_name]['classification_targets'][classification_target] = torch.tensor(decoder_outputs['classification_targets'][classification_target], dtype=torch.float32)
            for regression_target in decoder_outputs['regression_target']:
                if not torch.is_tensor(decoder_outputs['regression_target'][regression_target]):
                    target[decoder_name]['regression_target'][regression_target] = torch.tensor(decoder_outputs['regression_target'][regression_target], dtype=torch.float32)

        # move to device
        sample = sample.to(self.device)
        target = {decoder_name: {task: {param_name: param.to(self.device) for param_name, param in task_params.items()} for task, task_params in decoder_outputs.items()} for decoder_name, decoder_outputs in target.items()}

        return sample, target

In [11]:
dataset_name = "DAGDataset100_100_5"

In [12]:
dataset = DAGDataset(dataset_name)

In [ ]:
dataset.__getitem__(0)

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]], device='cuda:0'),
 {'Building Mass Decoder': {'classification_targets': {'Bm Base Shape': tensor([0., 1.], device='cuda:0')},
   'regression_target': {'Bm Size': tensor([0.2988, 0.7557, 0.6970], device='cuda:0')}},
  'Facade Decoder': {'classification_targets': {},
   'regression_target': {'Num Floors': tensor(0.5000, device='cuda:0'),
    'Num Windows Each Side': tensor(0.5000, device='cuda:0')}},
  'Floor Ledge Decoder': {'classification_targets': {'Has Floor Ledge': tensor([1., 0.], device='cuda:0')},
   'regression_target': {'Floor Ledge Size X': tensor(0.5630, device='cuda:0'),
    'Floor Ledge Size Z': tensor(0.2744, device='cuda:0'),
    'Floor Ledge Extrusion X': tensor(0.1379, device='cuda:0'),
    'Floor Ledge E

In [ ]:
# # Intentional overfit

# # split into train val and test
# train_size = int(0.8 * len(dataset))
# print(train_size)
# val_size = int(0.1 * len(dataset))
# print(val_size)
# test_size = len(dataset) - train_size - val_size
# print(test_size)
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# # Create DataLoader
# batch_size = 32
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

3
0
1


In [ ]:
# split into train val and test
train_size = int(0.8 * len(dataset))
print(train_size)
val_size = int(0.1 * len(dataset))
print(val_size)
test_size = len(dataset) - train_size - val_size
print(test_size)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
encoder = Encoder()

In [ ]:
# Load the ranges from the YAML file
with open(f'./datasets/{dataset_name}/ranges.yml', 'r') as file:
    ranges = yaml.safe_load(file)

# Create a mapping between parameter names and output sizes
parameter_output_mapping = {}
for decoder_name, param_specs in ranges.items():
    if param_specs['type'] == 'float':
        parameter_output_mapping[decoder_name] = 1  # 1 for scalar
    elif param_specs['type'] == 'int':
        parameter_output_mapping[decoder_name] = 1  # 1 for scalar
    elif param_specs['type'] == 'vector':
        parameter_output_mapping[decoder_name] = 3  # 3 for x, y, z
    elif param_specs['type'] == 'states':
        parameter_output_mapping[decoder_name] = len(param_specs['values'])
    elif param_specs['type'] == 'bool':
        parameter_output_mapping[decoder_name] = 2  # 2 for binary encoding

In [ ]:
# Load the decoders' params from the YAML file
with open(f'./datasets/{dataset_name}/decoders.yml', 'r') as file:
    decoders_params = yaml.safe_load(file)

decoders = nn.ModuleDict()
# initialize decoders with corresponding output tails
for decoder_name, param_names in decoders_params.items():
    classification_tails = {}
    regression_tails = {}
    for param_name in param_names:
        spec = ranges[param_name]
        # if type is bool or states, add to classification tails
        # if type is float, int or vector, add to regression tails
        if spec['type'] == 'bool' or spec['type'] == 'states':
            classification_tails[param_name] = parameter_output_mapping[param_name]
        else:
            regression_tails[param_name] = parameter_output_mapping[param_name]
    # add decoder to model
    decoders[decoder_name] = ParamAwareMultiTailDecoder(1024, classification_tails, regression_tails)

# decoders = nn.ModuleDict({
#     param_name: Decoder(4096, output_size)
#     for param_name, output_size in parameter_output_mapping.items()
# })

In [ ]:
model = EncoderDecoderModel(encoder, decoders)

In [ ]:
model

EncoderDecoderModel(
  (encoder): Encoder(
    (features): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): Linear(in_features=131072, out_features=1024, bias=True)
    )
  )
  (decoders): ModuleDict(
    (Building Mass Decoder): ParamAwareMultiTailDecoder(
      (fc1): Linear(in_features=1024, out_features=1024, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.5, inplace=False)
      (

In [ ]:
# Load the switches from the YAML file
with open(f'./datasets/{dataset_name}/switches.yml', 'r') as file:
    switches = yaml.safe_load(file)

In [ ]:
# Define loss function and optimizer
# for regression, use MSELoss, for classification, use CrossEntropyLoss
class EncDecsLoss(nn.Module):
    def __init__(self, decoders, switches_mapping: dict):
        super(EncDecsLoss, self).__init__()
        self.decoders = decoders
        self.switches_mapping = switches_mapping

    def forward(self, outputs, targets):
        loss = 0.0
        for decoder_name, decoder_output in outputs.items():
            loss += self.decoder_loss(decoder_output, targets[decoder_name])
        return loss

    def classification_loss(self, output, target):
        return nn.CrossEntropyLoss()(output, target)

    def regression_loss(self, output, target):
        return nn.MSELoss()(output, target)

    # def find_param_name_in(target, param_name):
    #     for decoder_name, decoder_outputs in target.items():
    #         for i, classification_target in enumerate(decoder_outputs['classification_targets']):
    #             if param_name ==
    #     return None

    def decoder_loss(self, decoder_output, target):
        classification_outputs = decoder_output[0]  # note that model outputs a tuple of list instead of dict of list
        regression_output = decoder_output[1]
        total_classification_loss = 0.0
        # if classification_outputs:
        for param_name, pred in classification_outputs.items():
            total_classification_loss += self.classification_loss(pred, target["classification_targets"][param_name])
            # TODO: should we add early termination for "Has" labels?
        # if regression_output:
        total_regression_loss = 0.0
        for param_name, pred in regression_output.items():
            regression_loss = self.regression_loss(pred, target["regression_target"][param_name])
            # use gt's 0 1 label to switch off the loss if needed
            switch_param_name = self.switches_mapping["Reversed Mapping"].get(param_name)
            if switch_param_name:
                switch_target = target["classification_targets"][switch_param_name]
                switch_index = torch.argmax(switch_target, dim=1)
                # make regression_loss same shape as switch_index
                regression_loss = torch.stack([regression_loss] * switch_index.size(0))
                regression_loss *= switch_index
                # average again
                regression_loss = torch.mean(regression_loss)
            total_regression_loss += regression_loss
        averaged_classification_loss = total_classification_loss / len(classification_outputs) if len(classification_outputs) > 0 else 0
        averaged_regression_loss = total_regression_loss / len(regression_output) if len(regression_output) > 0 else 0
        loss = averaged_classification_loss + averaged_regression_loss
        return loss

criterion = EncDecsLoss(decoders, switches)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device Conut: {torch.cuda.device_count()}")
print(f"Current CUDA Device: {torch.cuda.current_device()}")
print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA Available: True
CUDA Device Conut: 4
Current CUDA Device: 0
CUDA Device Name: NVIDIA GeForce GTX 1080 Ti


In [ ]:
# move model to device
model.to(device)

EncoderDecoderModel(
  (encoder): Encoder(
    (features): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): Linear(in_features=131072, out_features=1024, bias=True)
    )
  )
  (decoders): ModuleDict(
    (Building Mass Decoder): ParamAwareMultiTailDecoder(
      (fc1): Linear(in_features=1024, out_features=1024, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.5, inplace=False)
      (

In [ ]:
from tqdm import tqdm

epochs = 50
train_losses = []
val_losses = []

torch.manual_seed(0)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    num_batches = len(train_loader)
    progress_bar = tqdm(enumerate(train_loader), total=num_batches, desc=f"Epoch {epoch + 1}", leave=False)

    for i, data in progress_bar:
        inputs, targets = data

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        progress_bar.set_postfix({"training_loss": train_loss / (i + 1)})

    train_losses.append(train_loss / num_batches)

    # Validation
    model.eval()
    val_loss = 0.0
    num_batches_val = len(val_loader)

    with torch.no_grad():
        for i, data in enumerate(val_loader):
            inputs, targets = data

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_losses.append(val_loss / num_batches_val)

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}")

print("Finished Training")

# Save your trained model if needed
torch.save(model.state_dict(), "encDecModel.pth")


Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]/home/shuyuan/miniconda3/envs/dag/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/200, Training Loss: 5.318198204040527, Validation Loss: 5.124059200286865


Epoch 2/200, Training Loss: 5.231717109680176, Validation Loss: 4.71890115737915


Epoch 3/200, Training Loss: 4.863566875457764, Validation Loss: 4.301918983459473


Epoch 4/200, Training Loss: 4.017567157745361, Validation Loss: 3.923346996307373


...

Epoch 196/200, Training Loss: 2.2299771308898926, Validation Loss: 1.907730221748352


Epoch 197/200, Training Loss: 2.146874189376831, Validation Loss: 1.961092233657837


Epoch 198/200, Training Loss: 1.9773589372634888, Validation Loss: 1.9846410751342773


Epoch 199/200, Training Loss: 2.006676197052002, Validation Loss: 1.956526279449463


Epoch 200/200, Training Loss: 2.0131592750549316, Validation Loss: 1.9576191902160645
Finished Training


In [ ]:
# Test the model
model.eval()
test_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, targets = data
        # inputs, targets = inputs.to(device), targets.to(device)  # already moved to device in __getitem__
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
    print(f"Test loss: {test_loss / len(test_loader)}")
print("Finished Testing")

Test loss: 1.9576191902160645
Finished Testing


In [ ]:
outputs

{'Building Mass Decoder': ({'Bm Base Shape': tensor([[4.0746e-03, 9.9593e-01],
           [6.6665e-04, 9.9933e-01],
           [9.8423e-01, 1.5766e-02]], device='cuda:0')},
  {'Bm Size': tensor([[0.2117, 0.5559, 0.4739],
           [0.2136, 0.6243, 0.5328],
           [0.5412, 0.4837, 0.1513]], device='cuda:0')}),
 'Facade Decoder': ({},
  {'Num Floors': tensor([[0.5225],
           [0.5219],
           [0.5234]], device='cuda:0'),
   'Num Windows Each Side': tensor([[0.5306],
           [0.5324],
           [0.5318]], device='cuda:0')}),
 'Floor Ledge Decoder': ({'Has Floor Ledge': tensor([[9.9953e-01, 4.7015e-04],
           [1.2822e-04, 9.9987e-01],
           [2.1094e-03, 9.9789e-01]], device='cuda:0')},
  {'Floor Ledge Size X': tensor([[0.3526],
           [0.4790],
           [0.3921]], device='cuda:0'),
   'Floor Ledge Size Z': tensor([[0.2641],
           [0.2437],
           [0.2185]], device='cuda:0'),
   'Floor Ledge Extrusion X': tensor([[0.1203],
           [0.1338],
     

In [ ]:
targets

{'Building Mass Decoder': {'classification_targets': {'Bm Base Shape': tensor([[0., 1.],
           [0., 1.],
           [1., 0.]], device='cuda:0')},
  'regression_target': {'Bm Size': tensor([[0.2988, 0.7557, 0.6970],
           [0.2652, 0.7625, 0.6274],
           [0.9276, 0.7993, 0.0675]], device='cuda:0')}},
 'Facade Decoder': {'classification_targets': {},
  'regression_target': {'Num Floors': tensor([0.5000, 0.5000, 0.5000], device='cuda:0'),
   'Num Windows Each Side': tensor([0.5000, 0.5000, 0.5000], device='cuda:0')}},
 'Floor Ledge Decoder': {'classification_targets': {'Has Floor Ledge': tensor([[1., 0.],
           [0., 1.],
           [0., 1.]], device='cuda:0')},
  'regression_target': {'Floor Ledge Size X': tensor([0.5630, 0.8778, 0.4430], device='cuda:0'),
   'Floor Ledge Size Z': tensor([0.2744, 0.2707, 0.3260], device='cuda:0'),
   'Floor Ledge Extrusion X': tensor([0.1379, 0.1396, 0.2135], device='cuda:0'),
   'Floor Ledge Extrusion Z': tensor([0.3953, 0.4294, 0.3394